# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [17]:
import requests
import json
import logging
import pandas as pd
import azureml.core
import joblib

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Workspace & experiment

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoML-CapstoneProj-experiment-0'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: udc-machinelearning01
Azure region: eastus2
Subscription id: 7023dccf-8c31-41db-809e-9f442490bfe7
Resource group: rg-udc-ml-dev-eastus-01


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The heart_failure_clinical_records_dataset.cvs was acquired from Kaggle (Dataset from Davide Chicco, Giuseppe Jurman). 
https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data/data

About the data
age: Age of the patient
anaemia: Haemoglobin level of patient (Boolean)
creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
diabetes: If the patient has diabetes (Boolean)
ejection_fraction: Percentage of blood leaving the heart at each contraction
high_blood_pressure: If the patient has hypertension (Boolean)
platelets: Platelet count of blood (kiloplatelets/mL)
serum_creatinine: Level of serum creatinine in the blood (mg/dL)
serum_sodium: Level of serum sodium in the blood (mEq/L)
sex: Sex of the patient
smoking: If the patient smokes or not (Boolean)
time: Follow-up period (days)
DEATH_EVENT: If the patient deceased during the follow-up period (Boolean)

### Access the dataset

In [7]:
dataset = Dataset.get_by_name(ws, name = 'heart_failure_clinical_records_dataset')
dataset.take(10).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
5,90.0,1,47,0,40,1,204000.00,2.1,132,1,1,8,1
6,75.0,1,246,0,15,0,127000.00,1.2,137,1,0,10,1
7,60.0,1,315,1,60,0,454000.00,1.1,131,1,1,10,1
8,65.0,0,157,0,65,0,263358.03,1.5,138,0,0,10,1
9,80.0,1,123,0,35,1,388000.00,9.4,133,1,1,10,1


## Computer Cluster

In [8]:
# Personal Subcription - using existing cluster
cluster_name = "udc-ml-cluster-01"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    print('Creating a new cluster.')
    # 4 cores, 8GB RAM, 32GB storage
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_F4s_v2',
                                                          max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Creating a new cluster.
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric": "accuracy",
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=cluster,
    task="classification",
    training_data= dataset,
    label_column_name="DEATH_EVENT",
    featurization="auto",
    n_cross_validations= 5,
    **automl_settings)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-CapstoneProj-experiment-0,AutoML_5743bf0a-9b3d-44d9-b862-364e3a06e6a8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_5743bf0a-9b3d-44d9-b862-364e3a06e6a8',
 'target': 'udc-ml-cluster-01',
 'status': 'Completed',
 'startTimeUtc': '2024-04-10T22:51:02.602535Z',
 'endTimeUtc': '2024-04-11T00:13:43.014676Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udc-ml-cluster-01',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-CapstoneProj-experiment-0","subscription_id":"7023dccf-8c31-41db-809e-9f442490bfe7","resource_group":"rg-udc-ml-dev-eastus-01","workspace_name":"udc-machinelearning01","region":"eastus2","compute_target":"udc-ml-cluster-01","spark_service":null,"azure_service":"remote","many_models":false,"pipeli

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
# Retrieve your best automl model.
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

{'precision_score_macro': 0.8920738371818249, 'recall_score_micro': 0.8863841807909605, 'precision_score_micro': 0.8863841807909605, 'f1_score_micro': 0.8863841807909605, 'precision_score_weighted': 0.9020498920954232, 'recall_score_weighted': 0.8863841807909605, 'f1_score_macro': 0.8626367687648797, 'AUC_weighted': 0.910876661129568, 'accuracy': 0.8863841807909605, 'f1_score_weighted': 0.8827426716344096, 'weighted_accuracy': 0.9044080729594931, 'average_precision_score_macro': 0.8927321701978175, 'AUC_macro': 0.9108766611295682, 'log_loss': 0.38176963410810344, 'average_precision_score_weighted': 0.9192388787586282, 'AUC_micro': 0.9144207443582623, 'average_precision_score_micro': 0.9160376725191952, 'norm_macro_recall': 0.7188095238095238, 'recall_score_macro': 0.8594047619047618, 'balanced_accuracy': 0.8594047619047618, 'matthews_correlation': 0.7487447784810415, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_5743bf0a-9b3d-44d9-b862-364e3a06e6a8_43/accuracy_table', '

In [18]:
# Save the best model
joblib.dump(best_model, filename="./automl_bestmodel.pkl")

['./automl__bestmodel.pkl']

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [79]:
model = Model.register(workspace=ws,
                       model_name="AutoML_CapstoneProj_Exp0_BestModel", 
                       model_path="./automl__bestmodel.pkl",
                       description="Best AutoML model"
                      )

Registering model AutoML_CapstoneProj_Exp0_BestModel


In [80]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-CapstoneProj-experiment-0,AutoML_5743bf0a-9b3d-44d9-b862-364e3a06e6a8_43,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [29]:
# create inference config
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

In [81]:
interence_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2,
                                               memory_gb = 4,
                                               enable_app_insights = True,
                                               tags = {'type': 'automl'},
                                               description = 'Service for AutoML Best Model')

aci_service_name = 'heartfailurepred-service'
aci_service = Model.deploy(ws, aci_service_name, [model], interence_config, aciconfig)

aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads



2024-04-11 23:15:09+00:00 Creating Container Registry if not exists.
2024-04-11 23:15:09+00:00 Use the existing image.
2024-04-11 23:15:10+00:00 Submitting deployment to compute.
2024-04-11 23:15:15+00:00 Checking the status of deployment heartfailurepred-service.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


.
2024-04-11 23:17:16+00:00 Checking the status of inference endpoint heartfailurepred-service.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads



Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


TODO: In the cell below, print the logs of the web service and delete the service

In [82]:
# print the logs
print(aci_service.get_logs())

2024-04-11T23:17:08,070165800+00:00 - rsyslog/run 
2024-04-11T23:17:08,081525100+00:00 - gunicorn/run 
2024-04-11T23:17:08,087769000+00:00 | gunicorn/run | 
2024-04-11T23:17:08,094541200+00:00 | gunicorn/run | ###############################################
2024-04-11T23:17:08,101524800+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-04-11T23:17:08,101935500+00:00 - nginx/run 
2024-04-11T23:17:08,108628100+00:00 | gunicorn/run | ###############################################
2024-04-11T23:17:08,111277200+00:00 | gunicorn/run | 
2024-04-11T23:17:08,122636600+00:00 | gunicorn/run | 
2024-04-11T23:17:08,132140100+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-04-11T23:17:08,134963800+00:00 | gunicorn/run | 
2024-04-11T23:17:08,143635200+00:00 | gunicorn/run | 
2024-04-11T23:17:08,154613100+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

In [85]:
# delete the service
aci_service.delete()

Running
2024-04-12 03:53:08+00:00 Deleting service.
2024-04-12 03:53:11+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
